In [26]:
from Multi_head_crossattn import MultiHeadCrossAttention
from Multi_head_selfattn import MultiHeadSelfAttention
from feedforward import FeedForward
import torch.nn as nn
import torch.nn.functional as F
import torch

In [30]:
class TransformerDecoderLayer(nn.Module):
  def __init__(self,n_heads,batch_size,sequence_len, n_embedding):
    super().__init__()
    self.n_heads = n_heads
    self.batch_size = batch_size
    self.sequence_len = sequence_len
    self.n_embedding = n_embedding

    self.qkv_layer = nn.Linear(n_embedding,3*self.n_embedding)
    self.dropout1 = nn.Dropout(0.1)
    self.attention = MultiHeadSelfAttention()
    self.linear_layer = nn.Sequential(
            nn.Linear(n_embedding,2*n_embedding),
            nn.ReLU(),
            nn.Linear(2*n_embedding,n_embedding)
        )
    self.layer_norm1 = nn.LayerNorm(n_embedding)

    self.q_layer = nn.Linear(n_embedding,self.n_embedding)
    self.kv_layer = nn.Linear(n_embedding,2*self.n_embedding)
    self.cross_attention = MultiHeadCrossAttention(n_embedding)
    self.dropout2 = nn.Dropout(0.1)
    self.layer_norm2 = nn.LayerNorm(n_embedding)

    self.fcl = FeedForward(n_embedding)
    self.dropout3 = nn.Dropout(0.1)
    self.layer_norm3 = nn.LayerNorm(n_embedding)

  def forward(self,x,y,mask=None):
    residual = y
    y = self.qkv_layer(y)
    print(y.shape)
    y = y.view(self.batch_size,self.sequence_len,self.n_heads,3*self.n_embedding//self.n_heads)
    y = y.permute(0,2,1,3)
    y = self.attention(y)
    y= y.reshape(self.batch_size,self.sequence_len,self.n_embedding)
    y = self.dropout1(y)
    # y = y + residual
    y = self.layer_norm1(y+residual)

    residual = y
    q = self.q_layer(y)
    kv = self.kv_layer(x)
    q=q.view(self.batch_size,self.sequence_len,self.n_heads,self.n_embedding//self.n_heads)
    kv = kv.view(self.batch_size,self.sequence_len,self.n_heads,2*self.n_embedding//self.n_heads)
    q = q.permute(0,2,1,3)
    kv = kv.permute(0,2,1,3)
    y = self.cross_attention(q,kv,mask)
    y = y.reshape(self.batch_size,self.sequence_len,self.n_embedding)
    # y = nn.Linear(2*n_embedding,n_embedding)
    y = self.linear_layer(y)
    y = self.dropout2(y)
    # y = y + residual
    y = self.layer_norm2(y+residual)

    residual = y
    y = self.fcl(y)
    y = self.dropout3(y)
    # y = y + residual
    y = self.layer_norm3(y+residual)
    return y

class TransformerDecoder(nn.Module):
  def __init__(self,n_layers,n_heads,batch_size,sequence_len, n_embedding):
    super().__init__()
    self.layers = nn.ModuleList([TransformerDecoderLayer(n_heads,batch_size,sequence_len, n_embedding) for _ in range(n_layers)])

  def forward(self,x,y):
    for idx , layer in enumerate(self.layers):
      print(f'Layer : {idx+1}')
      y = layer(x,y)
    return y


In [31]:
n_heads = 8
batch_size = 4
sequence_len = 4
n_embedding = 512
n_layers = 5

qkv = torch.randn(batch_size,sequence_len,n_embedding*3)

qkv = qkv.reshape(batch_size,sequence_len,n_heads,3*n_embedding//n_heads)

qkv = qkv.permute(0,2,1,3)

In [32]:
decoder = TransformerDecoder(n_layers,n_heads,batch_size,sequence_len,n_embedding)
output = decoder(torch.randn(batch_size,sequence_len,n_embedding),torch.randn(batch_size,sequence_len,n_embedding))
print(output.shape)

TypeError: unsupported operand type(s) for +: 'TransformerDecoderLayer' and 'int'